## Creating the database
In this step, the goal is to establish a sales database in MySQL so I can populate data.

In [ ]:
# import all of the libraries needed in order to establish a data base
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:
# set up the mysql arguments to connect to the mysql server
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "password2025"

dst_dbname = "sales" # writing the data into this database

In [ ]:
# defining functions in order to get dataframes from the database and add tables to 
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")) # goes to col that you specify and turn it into a pkey

            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append') # append to existing df
    
    connection.close()

In [ ]:
# Creating the database, and to use it, switch the Connection Context
# this step allows me to create the sales database and allow me to add all of the date from the .sql files
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

Using the Table.sql and Insert.sql files, I will now begin to populate the database using MySQL. These database files originated from
https://github.com/aaronzguan/Online-Shopping-Cart-Database-Project/tree/master.

Once the data is populated, I selected everything in the comments table and exported the data as a json.

Similarly, I did the same thing for the Products table, but instead I exported the data as a csv.

## Reading in the Data

In this step, I will be reading in the data from a number of sources:
- JSON file
- CSV file
- MySQL Statements

Note: Lab 2c was run in order to populate a dim_date table. The only difference is that I had to change the ending date so it would populate
data until 2020

In [11]:
# importing all of the necessary packages for the next steps (note there are repeats from before)
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [13]:
# Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working
mysql_args = {
    "uid" : "root",
    "pwd" : "password2025",
    "hostname" : "localhost",
    "dbname" : "sales"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    #"user_name" : "root",
   # "password" : "Mongodb2016",
   # "cluster_name" : "cluster_name",
   # "cluster_subnet" : "xxxxx",
    "cluster_location" : "local", # "local"
    "db_name" : "sales_midterm"
}

In [15]:

print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.34
Running PyMongo Version: 4.10.1


In [17]:
# Define Functions for Getting Data From and Setting Data Into Databases
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [19]:
# Populate MongoDB with Source Data
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'midterm_data_json')

json_files = {
              "comments": 'Comments.json',
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files) 

In [21]:
#### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "comments"

df_comments = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)

In [23]:
### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the source (my laptop) Into DataFrames (from the folder with the csv file)
data_dir = os.path.join(os.getcwd(), 'midterm_data_csv')
data_file1 = os.path.join(data_dir, 'Product.csv')

df_products = pd.read_csv(data_file1, header=0, index_col=0)

In [25]:
# Extract Data from the Source Database Tables
src_dbname = "sales" # i had to specify this because initially we had no src_dbname, but now that the data is populated, we
# have a source database

sql_users = "SELECT * FROM sales.Users;"
df_users = get_dataframe(user_id, pwd, host_name, src_dbname, sql_users)

###
sql_seller = "SELECT * FROM sales.Seller;"
df_sellers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_seller)

###
sql_store = "SELECT * FROM sales.Store;"
df_stores = get_dataframe(user_id, pwd, host_name, src_dbname, sql_store)

###
sql_dim_date = "SELECT date_key, full_date FROM sales.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date

### Transforming the Data:

In these next steps, I will be editing the tables:
- One key step is to add an ever incrementing numeric value that will serve as the primary key
- Change any time values to date_keys instead so they can be easily sorted if needed
- Change any names of the columns so they are easily understood (ie: instead of id, have table_id)
- Making sure the data transformations are done correctly by viewing the first 2 rows

In [27]:
df_products = df_products.reset_index() # for the csv file, the product_id was being treated as a rowname and index so it originally did
# not show up unles you reset the index

df_products.rename(columns={"pid":"product_id"}, inplace=True) # rename the pid to product_id so it makes more sense
df_products.rename(columns={"name":"product_name"}, inplace=True) # rename the name colun because a few tables have a name variable
df_products.rename(columns={"sid":"store_id"}, inplace=True) # rename the sid to store_id so it makes more sense
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1)) 
# Inserted a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_products.head(2)

,product_key,product_id,store_id,brand,product_name,type,modelNumber,color,amount,price
0,1,1,8,Asus,ASUS Chromebook 11.6 laptop,laptop,C201PA-DS02,navy blue,10,262
1,2,2,10,Bose,Bose QuietComfort 35 wireless headphone,headphone,759944-0010,black,100,449


In [29]:
df_comments.rename(columns={"pid":"product_id"}, inplace=True)# rename the pid to product_id so it makes more sense
df_comments.rename(columns={"userid":"user_id"}, inplace=True) # rename the userid to user_id so it makes more sense
df_comments.insert(0, "comment_key", range(1, df_comments.shape[0]+1))
# Inserted a new column, with an ever-incrementing numeric value, to serve as the primary key.

# the next following steps are so I can change a date column in the comments dataframe to a date_key value
df_comments = df_comments.rename(columns={"creationTime" : "purchase_date"})
df_dim_comments_date = df_dim_date.rename(columns={"date_key" : "purchase_date_key", "full_date" : "purchase_date"})
df_dim_comments_date = df_dim_comments_date[["purchase_date_key", "purchase_date"]]
df_comments.purchase_date = df_comments.purchase_date.astype('datetime64[ns]').dt.date
df_comments = pd.merge(df_comments, df_dim_comments_date, on='purchase_date', how='left')
df_comments.drop(['purchase_date'], axis=1, inplace=True)

df_comments.head(2)

,comment_key,user_id,product_id,grade,content,purchase_date_key
0,1,12,3,4.7,Someone always has a better camera. That being...,20140322
1,2,12,2,4.3,I have been using it for a week now. For a sho...,20150530


In [31]:
df_users.rename(columns={"userid":"user_id"}, inplace=True) # rename userid to user_id so it makes more sense
df_users.insert(0, "user_key", range(1, df_users.shape[0]+1))
# Inserted a new column, with an ever-incrementing numeric value, to serve as the primary key.

df_users.head(2)

,user_key,user_id,name,phoneNumber
0,1,1,Leonard Carroll,809-902-4957
1,2,2,Sybill C. Kane,797-156-7733


In [33]:
df_stores.rename(columns={"sid":"store_id"}, inplace=True) #renamed sid to store_id so it made more sense
df_stores.insert(0, "store_key", range(1, df_stores.shape[0]+1))
# Inserted a new column, with an ever-incrementing numeric value, to serve as the primary key.

# Inserted a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_stores = df_stores.rename(columns={"startTime" : "start_date"})
df_dim_store_date = df_dim_date.rename(columns={"date_key" : "start_date_key", "full_date" : "start_date"})
df_dim_store_date = df_dim_store_date[["start_date_key", "start_date"]]
df_stores.start_date = df_stores.start_date.astype('datetime64[ns]').dt.date
df_stores = pd.merge(df_stores, df_dim_store_date, on='start_date', how='left')
df_stores.drop(['start_date'], axis=1, inplace=True)
df_stores.rename(columns={"name":"store_name"}, inplace=True)

df_stores.head(2)

,store_key,store_id,store_name,province,city,streetaddr,customerGrade,start_date_key
0,1,8,Choby Collection,Quebec,Montreal,No.2012 Sanguinet,4,20060530
1,2,10,BestBuy,British Columbia,Vancouver,No.20 ShineStreet,5,20020805


### Uploading dim_tables to sales database
In the next step, the tranformed data will now be uploaded to MySQL as new tables in the sales database.
These will be the tables we use to create the fact table and to author the queries.

In [35]:
#Loading the Transformed DataFrames into the New Data Warehouse by Creating New Tables
###
dataframe = df_products
table_name = 'dim_products'
primary_key = 'product_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)


###
dataframe = df_comments
table_name = 'dim_comments'
primary_key = 'comment_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

###
dataframe = df_users
table_name = 'dim_users'
primary_key = 'user_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

###
dataframe = df_stores
table_name = 'dim_stores'
primary_key = 'store_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

### Creating the Fact Table (a Fact Orders table that summarizes the orders or buyers)
- the first step will be to use the dim_tables to get the keys and the ids (this step will also make sure that our dim_tables were populated correctly)
- the next step will be to create the fact table my merging some of the data frames and adding the necessary keys
- the final step will be to write the table back into mysql

In [37]:
sql_dim_products = "SELECT product_key, product_id, store_id FROM sales.dim_products;"
df_dim_products = get_sql_dataframe(sql_dim_products, **mysql_args)
df_dim_products.head(2)

,product_key,product_id,store_id
0,1,1,8
1,2,2,10


In [39]:
sql_dim_comments = "SELECT comment_key, user_id, product_id FROM sales.dim_comments;"
df_dim_comments = get_sql_dataframe(sql_dim_comments, **mysql_args)
df_dim_comments.head(2)

,comment_key,user_id,product_id
0,1,12,3
1,2,12,2


In [41]:
sql_dim_users = "SELECT user_key, user_id FROM sales.dim_users;"
df_dim_users = get_sql_dataframe(sql_dim_users, **mysql_args)
df_dim_users.head(2)

,user_key,user_id
0,1,1
1,2,2


In [43]:
sql_dim_stores = "SELECT store_key, store_id FROM sales.dim_stores;"
df_dim_stores = get_sql_dataframe(sql_dim_stores, **mysql_args)
df_dim_stores.head(2)

,store_key,store_id
0,1,8
1,2,10


In [61]:
# in this step, I will make a fact_orders table (this table will summarize the products that each customer ordered)
# we will first make a dim_orders table that combines the information of the commenters or buyers with their personal information
df_dim_orders = pd.merge(df_dim_users, df_dim_comments, on='user_id', how = 'right')
df_dim_orders.drop('user_id', axis = 1, inplace=True) # since we have the key, we will not need the id variable any more

# merge 1: we will merge our orders with the products so we can see what each customer has ordered
df_fact_orders = pd.merge(df_dim_orders, df_products, on = "product_id", how = "left")


In [63]:
# this step exists to clean up the fact_orders table
df_fact_orders.drop('product_id', axis=1, inplace=True)
df_fact_orders.insert(0, "fact_order_key", range(1, df_fact_orders.shape[0] + 1)) # making a key for this table that is ever incrementing
ordered_columns = ['fact_order_key','user_key', 'comment_key', 'product_key',
                   'store_id', 'product_name','brand', 'type', 'amount', 'price', 'modelNumber', 'color']
df_fact_orders = df_fact_orders[ordered_columns]

df_fact_orders.head(2)

,fact_order_key,user_key,comment_key,product_key,store_id,product_name,brand,type,amount,price,modelNumber,color
0,1,12,1,3,39,Canon EOS Rebel T5,Canon,cameras,50,500,9126B003,black
1,2,12,2,2,10,Bose QuietComfort 35 wireless headphone,Bose,headphone,100,449,759944-0010,black


In [75]:
# writing this table back into mysql
dataframe = df_fact_orders
table_name = 'fact_purchase_orders'
primary_key = 'fact_order_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

### Author SQL Queries 
this is to show that the fact table is connected to the dim_tables and that everything works

In [117]:
sql_purchase_orders = """
    SELECT users.name AS Name, AVG(`comments`.grade) AS Average_Customer_Rating,
    SUM(`orders`.price) AS Total_Spent
    FROM `sales`.`fact_purchase_orders` AS orders
    INNER JOIN `sales`.`dim_users` AS users
    ON orders.user_key = users.user_key
    INNER JOIN `sales`.`dim_comments` as comments
    ON orders.comment_key = comments.comment_key
    GROUP BY users.name;
"""
df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders

,Name,Average_Customer_Rating,Total_Spent
0,Inga K. Gonzales,4.60,1662.0
1,Nevada Langley,4.75,634.0
2,Cora Collins,4.65,861.0
3,Tyrone D. Harvey,4.45,711.0
4,Alea V. Brewer,4.05,1476.0


In [121]:
sql_quantity = """
    SELECT stores.store_name AS Store_Name, AVG(`comments`.grade) AS Average_Total_Product_Rating,
    SUM(`orders`.amount) AS Total_Quantity_of_All_Products,
    SUM(orders.price * orders.amount) AS Total_Product_Worth
    FROM `sales`.`fact_purchase_orders` AS orders
    INNER JOIN `sales`.`dim_stores` AS stores
    ON orders.store_id = stores.store_id
    INNER JOIN `sales`.`dim_comments` as comments
    ON orders.comment_key = comments.comment_key
    GROUP BY stores.store_name;
"""
df_fact_quantity = get_sql_dataframe(sql_quantity, **mysql_args)
df_fact_quantity

,Store_Name,Average_Total_Product_Rating,Total_Quantity_of_All_Products,Total_Product_Worth
0,CoolGuy,4.70,50.0,25000.0
1,BestBuy,4.45,200.0,89800.0
2,American Eagle,4.60,220.0,97940.0
3,Choby Collection,4.35,120.0,54560.0
4,Your Favorite,4.80,40.0,28520.0
